In [1]:
import pydicom
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf

In [2]:
#Reading Files:

data_dir = "E:/RSNA/train"
patients = os.listdir(data_dir)
labels_df = pd.read_csv("E:/RSNA/train_labels.csv")
label=labels_df.loc[:,'MGMT_value']
scan = ["FLAIR","T1w","T1wCE","T2w"]
labels_df.head()

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1


In [4]:
i=0
for patient in patients[:]:
#     label = labels_df._get_value(patient,"")
    path = data_dir + '/'+ patient + "/" + scan[0]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    # print(len(slices),label[i],slices[i].pixel_array.shape)
    i+=1
    print(i)
print("Done")
print(len(slices))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
i

In [ ]:
# type(patient)
# os.listdir(path)
print(np.array(slices).shape)

In [5]:
np.array(slices[0].pixel_array)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [ ]:
type(slices[0].pixel_array)

In [6]:
#FUNCTIONS WE USE:
l=[1,2,3,4,5,6,7,8,9]
def chunk(l,n):
    for i in range(0,len(l),n):
        yield l[i:i+n]

# print(list(chunk(l,3)))       
        
def mean(l):
    return sum(l)/len(l)

def resize(layer,HM_slices):
    if(len(layer)<HM_slices and HM_slices-len(layer)>1):
        layer.append(layer[-1])
        return resize(layer,HM_slices)
    elif(len(layer)<HM_slices and HM_slices-len(layer)==1):
        layer.append(layer[-1])
        return layer
    else:
        return layer
    
    
    
def process_data(patients, label_df, IMG_PX_SIZE = 50,HM_slices=20, visual=False, i=0):
    path = data_dir + '/'+ patient + "/" + scan[scan_type]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    i+=1

       
    new_slices = []

    slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

    chunk_sizes =  math.ceil(len(slices)/HM_slices)

    for slice_chunk in chunk(slices, chunk_sizes):
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)

    new_slices=resize(new_slices,HM_slices)
    len(new_slices)
        
    if visual:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice,cmap='gray')
        #         y.imshow(each_slice)
        plt.show()
            
    if label[i]==1: label[i]=np.array([0,1])
    elif label[i]==0: label[i]=np.array([1,0])

    return np.array(new_slices), label [i]
        

## Let's Focus on only one Scan

In [7]:
IMG_PX_SIZE = 32
HM_slices=20
i=0
j=0 
much_data=[]
ps=[]
for patient in patients[:]:
    # ps=[]
    for scan_type in range(1):
        
        path = data_dir + '/'+ patient + "/" + scan[scan_type]
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    #     print(len(slices),label[i],slices[i].pixel_array.shape)
        i+=1

       
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        print(slices[0].shape)

        chunk_sizes =  math.ceil(len(slices)/HM_slices)
    #     print(chunk_sizes)

        for slice_chunk in chunk(slices, chunk_sizes):
    #         print(slice_chunk)
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)
        
        # new_slices=np.array(new_slices)

        # print(type(np.array(new_slices)))
        # print(new_slices.shape)

        new_slices=resize(new_slices,HM_slices)
#         print(len(new_slices))
        
        # if label[i]==1: label[i]=[0,1]
        # elif label[i]==0: label[i]=[1,0]


    ps.append(new_slices)
    new_slices=np.array(new_slices)
    Tlabel=np.array(label[0:10])
    label=np.array(label)
    print(new_slices.shape)
    print(patient)


    # much_data.append(ps)
# plt.show()
# print('I am done')
# print(len(much_data),len(ps))

(32, 32)
(20, 32, 32)
00000
(32, 32)
(20, 32, 32)
00002
(32, 32)
(20, 32, 32)
00003
(32, 32)
(20, 32, 32)
00005
(32, 32)
(20, 32, 32)
00006
(32, 32)
(20, 32, 32)
00008
(32, 32)
(20, 32, 32)
00009
(32, 32)
(20, 32, 32)
00011
(32, 32)
(20, 32, 32)
00012
(32, 32)
(20, 32, 32)
00014
(32, 32)
(20, 32, 32)
00017
(32, 32)
(20, 32, 32)
00018
(32, 32)
(20, 32, 32)
00019
(32, 32)
(20, 32, 32)
00020
(32, 32)
(20, 32, 32)
00021
(32, 32)
(20, 32, 32)
00022
(32, 32)
(20, 32, 32)
00024
(32, 32)
(20, 32, 32)
00025
(32, 32)
(20, 32, 32)
00026
(32, 32)
(20, 32, 32)
00028
(32, 32)
(20, 32, 32)
00030
(32, 32)
(20, 32, 32)
00031
(32, 32)
(20, 32, 32)
00032
(32, 32)
(20, 32, 32)
00033
(32, 32)
(20, 32, 32)
00035
(32, 32)
(20, 32, 32)
00036
(32, 32)
(20, 32, 32)
00043
(32, 32)
(20, 32, 32)
00044
(32, 32)
(20, 32, 32)
00045
(32, 32)
(20, 32, 32)
00046
(32, 32)
(20, 32, 32)
00048
(32, 32)
(20, 32, 32)
00049
(32, 32)
(20, 32, 32)
00052
(32, 32)
(20, 32, 32)
00053
(32, 32)
(20, 32, 32)
00054
(32, 32)
(20, 32, 32

## TEST SET

In [20]:
IMG_PX_SIZE = 32
HM_slices=20
data_dir = "E:/RSNA/test"
patients = os.listdir(data_dir)
i=0
j=0 
much_data=[]
Testps=[]
for patient in patients[:]:
    # ps=[]
    for scan_type in range(1):
        
        path = data_dir + '/'+ patient + "/" + scan[scan_type]
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    #     print(len(slices),label[i],slices[i].pixel_array.shape)
        i+=1

       
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        print(slices[0].shape)

        chunk_sizes =  math.ceil(len(slices)/HM_slices)
    #     print(chunk_sizes)

        for slice_chunk in chunk(slices, chunk_sizes):
    #         print(slice_chunk)
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)
        
        # new_slices=np.array(new_slices)

        # print(type(np.array(new_slices)))
        # print(new_slices.shape)

        new_slices=resize(new_slices,HM_slices)
#         print(len(new_slices))
        
        # if label[i]==1: label[i]=[0,1]
        # elif label[i]==0: label[i]=[1,0]


    Testps.append(new_slices)
    new_slices=np.array(new_slices)
    Tlabel=np.array(label[0:10])
    print(new_slices.shape)
    print(patient)

(32, 32)
(20, 32, 32)
00001
(32, 32)
(20, 32, 32)
00013
(32, 32)
(20, 32, 32)
00015
(32, 32)
(20, 32, 32)
00027
(32, 32)
(20, 32, 32)
00037
(32, 32)
(20, 32, 32)
00047
(32, 32)
(20, 32, 32)
00079
(32, 32)
(20, 32, 32)
00080
(32, 32)
(20, 32, 32)
00082
(32, 32)
(20, 32, 32)
00091
(32, 32)
(20, 32, 32)
00114
(32, 32)
(20, 32, 32)
00119
(32, 32)
(20, 32, 32)
00125
(32, 32)
(20, 32, 32)
00129
(32, 32)
(20, 32, 32)
00135
(32, 32)
(20, 32, 32)
00145
(32, 32)
(20, 32, 32)
00153
(32, 32)
(20, 32, 32)
00161
(32, 32)
(20, 32, 32)
00163
(32, 32)
(20, 32, 32)
00174
(32, 32)
(20, 32, 32)
00181
(32, 32)
(20, 32, 32)
00182
(32, 32)
(20, 32, 32)
00190
(32, 32)
(20, 32, 32)
00200
(32, 32)
(20, 32, 32)
00208
(32, 32)
(20, 32, 32)
00213
(32, 32)
(20, 32, 32)
00229
(32, 32)
(20, 32, 32)
00252
(32, 32)
(20, 32, 32)
00256
(32, 32)
(20, 32, 32)
00264
(32, 32)
(20, 32, 32)
00287
(32, 32)
(20, 32, 32)
00307
(32, 32)
(20, 32, 32)
00323
(32, 32)
(20, 32, 32)
00333
(32, 32)
(20, 32, 32)
00335
(32, 32)
(20, 32, 32

In [11]:
ps=np.array(ps)
ps = np.expand_dims(ps, axis=-1)
ps.shape

(585, 20, 32, 32, 1)

In [21]:
Testps=np.array(Testps)
Testps = np.expand_dims(Testps, axis=-1)
Testps.shape

(87, 20, 32, 32, 1)

In [12]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
'''from scipy.ndimage import zoom
IMG_PX_SIZE = 250
i=0
j=0 
for patient in patients[:1]:
    path = data_dir + '/'+ patient + "/" + scan[0]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    i+=1
    print(len(slices))
    HM_slices=20
#     new_slices = []
    factor = len(slices)/20
    new_slices = zoom(np.array(slices),(1,1,factor))
    print(type(np.array(slices)))
    print(np.array(slices).shape)
      
#     for slice_chunk in chunk(slices, chunk_sizes):
#         slice_chunk = list(map(mean,zip(*slice_chunk)))
#         new_slices.append(slice_chunk)'''

In [18]:
# sample_shape = (32, 32, 20, 1)
#MODEL
model = Sequential()
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(20,32,32,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 18, 30, 30, 64)    1792      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 9, 15, 15, 64)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 15, 15, 64)     256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 15, 15, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 13, 13, 32)     55328     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 3, 6, 6, 32)       0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 3, 6, 6, 32)      

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label = np.array(label)
label_encode_x = OneHotEncoder(sparse=False)
label=label.reshape(585,1)
label=label_encode_x.fit_transform(label)

In [10]:
# len(new_slices)
label


array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [ ]:
# Fit data to model
model.fit(x=ps, y=label, batch_size=1,validation_split=0.3,epochs=50,verbose=1)

In [ ]:
# tf.config.experimental_run_functions_eagerly(True)

In [ ]:
# b = np.random.randn(10,20,32,32,1)

In [ ]:
# b.shape

In [ ]:
# model(b)

In [ ]:
# ps = np.expand_dims(ps, axis=-1)

In [ ]:
# ps.shape

In [22]:
predictions = model.predict(x=Testps,verbose=0)


C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


array([[0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.05688084e-35],
       [1.00000000e+00, 0.00000000e+00],
       [2.96366393e-14, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.43526649e-16, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.000000